
In your laboratory they were following the diet of a single person and making metabolomics
of biological samples extracted to this person at a fixed time post eating (the file with the
data you will find it in my github). They propose the following model (very simple) model:

**ATP = a * Proteins + b * Carbohydrates + c \* Fats + ε**

They ask you to find the best possible values from **a, b** and **c**, and say which is the extent of
error that this model have.

**Important:**  You can only use pandas and plotting libraries.

## 0. Import pandas and plotting libraries

In [11]:
import pandas as pd
import matplotlib.pyplot as plt

## 1. Read the data

(Found in diet_and_atp.tsv, that I got from: https://github.com/leandroradusky/AdvancedStatistics/blob/master/data)

In [12]:
data_table = pd.read_csv("./diet_and_atp.tsv", sep='\t')
data_table.head()

,Protein,Carbohydrates,Fat,ATP
0,1.95,6.92,0.02,9.44
1,0.64,2.65,0.15,4.04
2,1.82,0.98,0.21,3.72
3,1.80,4.69,0.23,7.54
4,0.93,4.22,0.02,5.70


## 2. Defining the function that should be minimized

In [195]:
def ATP_prediction_error(a,b,c, data = data_table):
    """For a given paramaters: a, b, and c, returns the sum squared of ε =ATP-(a*Proteins+b*Carbohydrates+c*Fats) """
    SSE = 0 #sum squared error
    for data_row in range(len(data)):
        Error = data["ATP"][data_row] - \
          (a*data["Protein"][data_row] + b*data["Carbohydrates"][data_row] + c*data["Fat"][data_row])
        SSE += Error**2
    return SSE


# If you like less lines cause "totally cooler", this does the exact same thing in a really large line

#ATP_prediction_error = lambda a, b, c, data = data_table : sum([(data["ATP"][i]-(a*data["Protein"][i]+b*data["Carbohydrates"][i]+c*data["Fat"][i]))**2 for i in range(len(data))])

## 3. Defining a function to get random numbers using pandas

In [103]:
def random(n, a,b=0):
    """Returns n random integers between a and b using pandas."""
    return(list(pd.DataFrame([i for i in range(min((a,b)),max((a,b)))]).sample(n=n, replace=1)[0]))

# In one line would be the same without a clear place to include an explanation:

random = lambda n, a, b=0 : list(pd.DataFrame([i for i in range(min((a,b)),max((a,b)))]).sample(n=n, replace=1)[0])

## 3. Markov Chain Monte Carlo

In [264]:
def Markov_Chain_Monte_Carlo(function_to_minimize, n_args, data = data_table, starts = 10, iterations=100,\
                             param_range=(0,4) ):
    """Uses a Markov Chain Monte Carlo aproach to explore the function_to_minimize and returns the 
    arguments in witch it has find the minimum value
    
    It repeats the aproach "starts" times doing "iterations" steps on each of them. On each repeat, the paramaters are 
    set randomly to an integer within the "param_range" """
    
    mimimum_values = [1]*n_args
    minimum_found = function_to_minimize(*mimimum_values, data = data)
    for start in range(starts):
        current_values = random(n_args, *param_range)
        cur_val = function_to_minimize(*current_values, data = data)
        for iterat in range(iterations):
            step = 1/(1+iterat/20)              # Reducing the length of the step when already near a minimum allows 
            in_var = random(1, n_args)[0]       # to find a more exat locatin 
            #print(step)
            negative = random(1,2)[0]
            
            new_values = current_values
            new_values[in_var] = new_values[in_var] + step*(1-2*negative)

            new_val = function_to_minimize(*new_values, data = data)
            
            if new_val < cur_val:
                
                cur_val = new_val
                current_values = new_values
                #print("hello", cur_val, current_values)
                
                if new_val < minimum_found:
                    #print("new found!", cur_val)
                    print(new_val, cur_val, new_values)
                    minimum_found = new_val
                    mimimum_values = [i for i in new_values] # force copy without using library copy
            
            else:
                if random(1,101)[0] > ((new_val - cur_val)/(cur_val))*100:
                    #print("         hey      ",new_val, cur_val, ((new_val - cur_val)/(cur_val))*100)
                    cur_val = new_val
                    current_values = new_values
    return(minimum_found, mimimum_values)
                
    

In [265]:
Markov_Chain_Monte_Carlo(ATP_prediction_error, 3, starts = 100, iterations=50)

18.123400000000007 18.123400000000007 [1, 1.0, 2]
13.79389999999999 13.79389999999999 [1.0, 1, 3]
12.327917170594176 12.327917170594176 [1.0, 1.0871447393186522, 1]
10.263412927856216 10.263412927856216 [1.0, 1.0871447393186522, 1.8333333333333335]


(10.263412927856216, [1.0, 1.0871447393186522, 1.8333333333333335])